# Лабораторная работа №2

Импортируем нужные библиотеки и объявляем нужные функции

In [1]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import mlflow
from sklearn.metrics import recall_score, precision_score, f1_score
from sklearn.preprocessing import PolynomialFeatures, KBinsDiscretizer
from sklearn.feature_selection import SequentialFeatureSelector 
from sklearn.model_selection import  GridSearchCV
from sklearn.metrics import make_scorer

/Users/alexandr/MPEI/IIS/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Загружаем очищенные данные

In [2]:
with open("../data/new_clean_dataset.pkl", "rb") as f:
    df = pickle.load(f)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1935 entries, 1 to 1999
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   battery_power       1935 non-null   int64   
 1   blue                1935 non-null   category
 2   clock_speed         1935 non-null   float64 
 3   dual_sim            1935 non-null   category
 4   fc                  1935 non-null   int64   
 5   four_g              1935 non-null   category
 6   int_memory          1935 non-null   int64   
 7   m_dep               1935 non-null   float64 
 8   mobile_wt           1935 non-null   int64   
 9   n_cores             1935 non-null   int64   
 10  pc                  1935 non-null   int64   
 11  px_height           1935 non-null   int64   
 12  px_width            1935 non-null   int64   
 13  ram                 1935 non-null   int64   
 14  sc_h                1935 non-null   int64   
 15  sc_w                1935 non-null   int64  

Разрделяем датасет на две выборки: обучающую и тестовую в соотношении 25% и 75%

In [3]:
X = df.drop('price_range', axis=1)
y = df['price_range']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.75)

Выделяем числовые и категориальные признаки

In [4]:
numeric_features = df.select_dtypes(exclude="category").columns.drop('price_range')
categorical_features = df.select_dtypes('category').columns

Pipepline

In [5]:
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', StandardScaler(), numeric_features),
        ('categorical', TargetEncoder(), categorical_features)
])
pipeline_baseline = Pipeline([
    ('transform', preprocessor),
    ('classification', RandomForestClassifier())
])

Baseline-модель

In [6]:
estimator = pipeline_baseline.fit(X_train, y_train)
predictions = estimator.predict(X_test)
estimator

Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('numeric', StandardScaler(),
                                                  Index(['battery_power', 'clock_speed', 'fc', 'int_memory', 'm_dep',
       'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'ram', 'sc_h',
       'sc_w', 'talk_time', 'battery_efficiency', 'screen_size'],
      dtype='object')),
                                                 ('categorical',
                                                  TargetEncoder(),
                                                  Index(['blue', 'dual_sim', 'four_g', 'three_g', 'touch_screen', 'wifi'], dtype='object'))])),
                ('classification', RandomForestClassifier())])

Mlflow

In [7]:
def calc_metrics(y_test, predictions, average='weighted'):
    metrics = {}
    metrics["recall"] = recall_score(y_test, predictions, average=average)   
    metrics["precision"] = precision_score(y_test, predictions, average=average)
    metrics["f1"] = f1_score(y_test, predictions, average=average)
    return metrics

In [8]:
TARGET_HOST = "127.0.0.1"
TARGET_PORT = 5000
TRACKING_URI = f"http://{TARGET_HOST}:{TARGET_PORT}"
REGISTRY_URI = TRACKING_URI

mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_registry_uri(REGISTRY_URI)

In [9]:
EXPERIMENT_NAME = 'MobilePrice Classification'
RUN_NAME = "baseline model"
REGISTRY_MODEL_NAME = "estate_model_rf"

signature = mlflow.models.infer_signature(model_input=X_train.head(5))
input_example = X_train.head(5)
req_file = '../requirements.txt'
params_dict = pipeline_baseline.get_params()

/Users/alexandr/MPEI/IIS/.venv/lib/python3.9/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [10]:
experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id 
    mlflow.sklearn.log_model(pipeline_baseline, 
                             artifact_path="models",
                             signature=signature,
                             input_example=input_example,
                             pip_requirements=req_file
                             )
    mlflow.log_metrics(calc_metrics(y_test, predictions))
    mlflow.log_params(params_dict)

run = mlflow.get_run(run_id) 
assert (run.info.status =='FINISHED')

2025/11/19 23:17:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/19 23:17:42 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - matplotlib (current: 3.9.2, required: matplotlib==3.10.6)
 - pandas (current: 2.2.2, required: pandas==2.3.2)
 - scikit-learn-1.7.2 (current: uninstalled, required: scikit-learn-1.7.2)
 - numpy (current: 2.0.2, required: numpy==1.26.4)
 - mlflow (current: 3.1.4, required: mlflow==2.16)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/11/19 23:17:42 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - matplotlib (current: 3.9.2, required: matplotlib==3.10.6)
 - pandas (current: 2.2.2, required: pandas=

🏃 View run baseline model at: http://127.0.0.1:5000/#/experiments/1/runs/daa2a506431d42a980ed50c8ad825c4b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


Сгенерируем новые признаки

In [11]:
def generate_features_polynomial(data, colname_template, transformer, init_features):
    new_data = transformer.fit_transform(data[init_features])
    new_features = [colname_template.format(i) for i in range(new_data.shape[1])]
    data[new_features] = new_data
    return data

In [12]:
X_train_fe_sklearn = X_train.copy()
X_test_fe_sklearn = X_test.copy()
poly_features_init = ['sc_h', 'sc_w']
kbins_features_init = ['battery_power', 'n_cores']


X_train_fe_sklearn = generate_features_polynomial(
    X_train_fe_sklearn, 'polynomial_{}', PolynomialFeatures(degree=2), poly_features_init
)

X_test_fe_sklearn = generate_features_polynomial(
    X_test_fe_sklearn, 'polynomial_{}', PolynomialFeatures(degree=2), poly_features_init
)

#X_train_fe_sklearn[poly_features_init] = X_train_fe_sklearn[poly_features_init].astype('float128')
#X_test_fe_sklearn[poly_features_init] = X_test_fe_sklearn[poly_features_init].astype('float128')

new_data = KBinsDiscretizer(n_bins=3).fit_transform(X_train_fe_sklearn[kbins_features_init])
X_train_fe_sklearn[[f'kbins_{i}' for i in range(new_data.shape[1])]] = new_data.todense()

new_data = KBinsDiscretizer(n_bins=3).fit_transform(X_test_fe_sklearn[kbins_features_init])
X_test_fe_sklearn[[f'kbins_{i}' for i in range(new_data.shape[1])]] = new_data.todense()

with open('../mlflow/new_feature_cols.txt', 'w') as f:
    print(*X_train_fe_sklearn.columns, sep=',', file=f)

pipeline_new_features = Pipeline([
    ('preprocessor', ColumnTransformer([
        (
            'numeric',
            StandardScaler(),
            X_train_fe_sklearn.select_dtypes(exclude='category').columns
        ),
        ('categorical', TargetEncoder(), categorical_features)])),
    ('classifier', RandomForestClassifier())
])
estimator = pipeline_new_features.fit(X_train_fe_sklearn, y_train)
display(estimator)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric', StandardScaler(),
                                                  Index(['battery_power', 'clock_speed', 'fc', 'int_memory', 'm_dep',
       'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'ram', 'sc_h',
       'sc_w', 'talk_time', 'battery_efficiency', 'screen_size',
       'polynomial_0', 'polynomial_1', 'polynomial_2', 'polynomial_3',
       'polynomial_4', 'polynomial_5', 'kbins_0', 'kbins_1', 'kbins_2',
       'kbins_3', 'kbins_4', 'kbins_5'],
      dtype='object')),
                                                 ('categorical',
                                                  TargetEncoder(),
                                                  Index(['blue', 'dual_sim', 'four_g', 'three_g', 'touch_screen', 'wifi'], dtype='object'))])),
                ('classifier', RandomForestClassifier())])

In [13]:
predictions = estimator.predict(X_test_fe_sklearn)

In [14]:
RUN_NAME = "new_features"
input_example = X_train_fe_sklearn.head(5)
signature = mlflow.models.infer_signature(model_input=input_example)

params_dict = pipeline_new_features.get_params()

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id 
    mlflow.sklearn.log_model(pipeline_new_features, 
                             artifact_path="models",
                             signature=signature,
                             input_example=input_example,
                             pip_requirements=req_file
                             )
    mlflow.log_metrics(calc_metrics(y_test, predictions))
    mlflow.log_artifact('../mlflow/new_feature_cols.txt')
    mlflow.log_params(params_dict)

run = mlflow.get_run(run_id) 
assert (run.info.status =='FINISHED')

/Users/alexandr/MPEI/IIS/.venv/lib/python3.9/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/11/19 23:17:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/19 23:17:53 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependenc

🏃 View run new_features at: http://127.0.0.1:5000/#/experiments/1/runs/a33c3afb2f4c4d9784d6012d584a71fd
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


Отбор важных признаков

In [15]:
classifier_main_features = RandomForestClassifier()
selector = SequentialFeatureSelector(
    classifier_main_features, n_features_to_select=5, direction='forward'
)

selector.fit(X_train_fe_sklearn, y_train)

idx = selector.get_support(indices=True)
with open('../mlflow/main_features.txt', 'w') as f:
    print(*idx, sep=',', file=f)
    print(*X_train_fe_sklearn.columns[idx], sep=',', file=f)

print('main features:', *X_train_fe_sklearn.columns[idx])

pipeline_main_features = Pipeline([
    ('preprocessor', ColumnTransformer([
        (
            'numeric', StandardScaler(),
            X_train_fe_sklearn.select_dtypes(exclude='category').columns
        ),
        ('categorical', TargetEncoder(), categorical_features)])),
    ('selection', selector),
    ('classifier', classifier_main_features)
])
estimator = pipeline_main_features.fit(X_train_fe_sklearn, y_train)
display(estimator)
predictions = estimator.predict(X_test_fe_sklearn)

main features: battery_power px_height px_width ram kbins_2


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric', StandardScaler(),
                                                  Index(['battery_power', 'clock_speed', 'fc', 'int_memory', 'm_dep',
       'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'ram', 'sc_h',
       'sc_w', 'talk_time', 'battery_efficiency', 'screen_size',
       'polynomial_0', 'polynomial_1', 'polynomial_2', 'polynomial_3',
       'polynomial_4', 'polynomial_5', 'kbins_0', 'kbins_1', 'kbins_2',
       'kbins_3', 'kbins_4', 'kbins_5'],
      dtype='object')),
                                                 ('categorical',
                                                  TargetEncoder(),
                                                  Index(['blue', 'dual_sim', 'four_g', 'three_g', 'touch_screen', 'wifi'], dtype='object'))])),
                ('selection',
                 SequentialFeatureSelector(estimator=RandomForestClassifier(),
                                           n_features_to_select=5)),
                ('classifier', RandomForestClassifier())])

In [16]:
RUN_NAME = "main_features"
input_example = X_train_fe_sklearn.iloc[:, idx].head(5)
signature = mlflow.models.infer_signature(model_input=input_example)

params_dict = pipeline_main_features.get_params()

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    # получаем уникальный идентификатор запуска эксперимента
    run_id = run.info.run_id 
    mlflow.sklearn.log_model(pipeline_main_features, 
                             artifact_path="models",
                             signature=signature,
                             input_example=input_example,
                             pip_requirements=req_file
                             )
    mlflow.log_metrics(calc_metrics(y_test, predictions))
    mlflow.log_artifact('../mlflow/main_features.txt')
    mlflow.log_params(params_dict)

run = mlflow.get_run(run_id) 
assert (run.info.status =='FINISHED')

/Users/alexandr/MPEI/IIS/.venv/lib/python3.9/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/11/19 23:22:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/19 23:22:01 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependenc

🏃 View run main_features at: http://127.0.0.1:5000/#/experiments/1/runs/c1da6f53a49143ad848b5c017bbb696b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


Настраиваем параметры для оптимальной модели

In [20]:
param_grid = {
    'classifier__n_estimators': [50, 100], #, 200, 300], 
    'classifier__max_depth': [None, 10, 15, 20, 25, 30],
    'classifier__max_features': [i/10 for i in range(1,10)],
}

gs = GridSearchCV(
    pipeline_main_features, 
    param_grid, 
    cv=3,
    scoring=make_scorer(f1_score, average='weighted'),
    n_jobs=-1,
    verbose=2
)

gs.fit(X_train_fe_sklearn, y_train)
print("Лучшие гиперпараметры:", gs.best_params_)
print("Лучшее значение f1-score:", gs.best_score_)
print("Лучшая модель:", gs.best_estimator_)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
[CV] END classifier__max_depth=None, classifier__max_features=0.1, classifier__n_estimators=50; total time= 2.7min
[CV] END classifier__max_depth=None, classifier__max_features=0.1, classifier__n_estimators=100; total time= 2.7min
[CV] END classifier__max_depth=None, classifier__max_features=0.1, classifier__n_estimators=50; total time= 2.7min
[CV] END classifier__max_depth=None, classifier__max_features=0.2, classifier__n_estimators=50; total time= 2.7min
[CV] END classifier__max_depth=None, classifier__max_features=0.1, classifier__n_estimators=100; total time= 2.7min
[CV] END classifier__max_depth=None, classifier__max_features=0.2, classifier__n_estimators=50; total time= 2.7min
[CV] END classifier__max_depth=None, classifier__max_features=0.2, classifier__n_estimators=100; total time= 2.7min
[CV] END classifier__max_depth=None, classifier__max_features=0.1, classifier__n_estimators=100; total time= 2.7min
[CV] END clas

In [21]:
print("Лучшие гиперпараметры:", gs.best_params_)
print("Лучшее значение f1-score:", gs.best_score_)
print("Лучшая модель:", gs.best_estimator_)

Лучшие гиперпараметры: {'classifier__max_depth': 15, 'classifier__max_features': 0.6, 'classifier__n_estimators': 100}
Лучшее значение f1-score: 0.9131819259417293
Лучшая модель: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric', StandardScaler(),
                                                  Index(['battery_power', 'clock_speed', 'fc', 'int_memory', 'm_dep',
       'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'ram', 'sc_h',
       'sc_w', 'talk_time', 'battery_efficiency', 'screen_size',
       'polynomial_0', 'polynomial_1', 'polynomial_2', 'polynomial_3',
       'polynomial_4', 'po...s_1', 'kbins_2',
       'kbins_3', 'kbins_4', 'kbins_5'],
      dtype='object')),
                                                 ('categorical',
                                                  TargetEncoder(),
                                                  Index(['blue', 'dual_sim', 'four_g', 'three_g', 'touch_screen', 'wifi'], dtype='objec

In [22]:
classifier_optimized = RandomForestClassifier(max_depth=None, max_features=0.1, n_estimators=50)

pipeline_optimized = Pipeline([
    ('preprocessor', ColumnTransformer([
        (
            'numeric', StandardScaler(),
            X_train_fe_sklearn.select_dtypes(exclude='category').columns
        ),
        ('categorical', TargetEncoder(), categorical_features)])),
    ('selection', selector),
    ('classifier', classifier_optimized)
])
estimator = pipeline_optimized.fit(X_train_fe_sklearn, y_train)
display(estimator)
predictions = estimator.predict(X_test_fe_sklearn)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric', StandardScaler(),
                                                  Index(['battery_power', 'clock_speed', 'fc', 'int_memory', 'm_dep',
       'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'ram', 'sc_h',
       'sc_w', 'talk_time', 'battery_efficiency', 'screen_size',
       'polynomial_0', 'polynomial_1', 'polynomial_2', 'polynomial_3',
       'polynomial_4', 'po...', 'kbins_2',
       'kbins_3', 'kbins_4', 'kbins_5'],
      dtype='object')),
                                                 ('categorical',
                                                  TargetEncoder(),
                                                  Index(['blue', 'dual_sim', 'four_g', 'three_g', 'touch_screen', 'wifi'], dtype='object'))])),
                ('selection',
                 SequentialFeatureSelector(estimator=RandomForestClassifier(),
                                           n_features_to_select=5)),
                ('classifier',
                 RandomForestClassifier(max_features=0.1, n_estimators=50))])

In [23]:
RUN_NAME = "best hyperparams"
input_example = X_train_fe_sklearn.head(5)
signature = mlflow.models.infer_signature(model_input=input_example)

params_dict = pipeline_main_features.get_params()

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    # получаем уникальный идентификатор запуска эксперимента
    run_id = run.info.run_id 
    mlflow.sklearn.log_model(pipeline_main_features, 
                             artifact_path="models",
                             signature=signature,
                             input_example=input_example,
                             pip_requirements=req_file
                             )
    mlflow.log_metrics(calc_metrics(y_test, predictions))
    mlflow.log_params(params_dict)

run = mlflow.get_run(run_id) 
assert (run.info.status =='FINISHED')

/Users/alexandr/MPEI/IIS/.venv/lib/python3.9/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/11/20 10:50:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/20 10:50:42 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependenc

🏃 View run best hyperparams at: http://127.0.0.1:5000/#/experiments/1/runs/df6b8c549641434c89506d00c84c430e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


Обучаем модель на всей выборке

In [24]:
X_fe_sklearn = pd.concat([X_train_fe_sklearn, X_test_fe_sklearn])
y = pd.concat([y_train, y_test])

In [25]:
pipeline_all_data= Pipeline([
    ('preprocessor', ColumnTransformer([
        (
            'numeric', StandardScaler(),
            X_fe_sklearn.select_dtypes(exclude='category').columns
        ),
        ('categorical', TargetEncoder(), categorical_features)])),
    ('selection', selector),
    ('classifier', classifier_main_features)
])

estimator = pipeline_main_features.fit(X_fe_sklearn, y)
display(estimator)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric', StandardScaler(),
                                                  Index(['battery_power', 'clock_speed', 'fc', 'int_memory', 'm_dep',
       'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'ram', 'sc_h',
       'sc_w', 'talk_time', 'battery_efficiency', 'screen_size',
       'polynomial_0', 'polynomial_1', 'polynomial_2', 'polynomial_3',
       'polynomial_4', 'polynomial_5', 'kbins_0', 'kbins_1', 'kbins_2',
       'kbins_3', 'kbins_4', 'kbins_5'],
      dtype='object')),
                                                 ('categorical',
                                                  TargetEncoder(),
                                                  Index(['blue', 'dual_sim', 'four_g', 'three_g', 'touch_screen', 'wifi'], dtype='object'))])),
                ('selection',
                 SequentialFeatureSelector(estimator=RandomForestClassifier(),
                                           n_features_to_select=5)),
                ('classifier', RandomForestClassifier())])

In [26]:
RUN_NAME = "all_dataset"
input_example = X_fe_sklearn.iloc[:, idx].head(5)
signature = mlflow.models.infer_signature(model_input=input_example)

params_dict = pipeline_all_data.get_params()

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id 
    mlflow.sklearn.log_model(pipeline_all_data, 
                             artifact_path="models",
                             signature=signature,
                             input_example=input_example,
                             pip_requirements=req_file
                             )
    mlflow.log_artifact('../mlflow/main_features.txt')
    mlflow.log_params(params_dict)

run = mlflow.get_run(run_id) 
assert (run.info.status =='FINISHED')

/Users/alexandr/MPEI/IIS/.venv/lib/python3.9/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/11/20 11:29:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/20 11:29:20 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependenc

🏃 View run all_dataset at: http://127.0.0.1:5000/#/experiments/1/runs/5ad9c54839cb46929012f2120fa83229
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
